In [1]:
# import boto3
# import findspark
# findspark.init()
# from pyspark.sql import functions as F

## TODO: Get the data from Kafka topic
# d = [{'uid': '123', 'evttype': 'click', 'segment': 'clothing'},{'uid': '123', 'evttype': 'click', 'segment': 'sports'},{'uid': '122', 'evttype': 'click', 'segment': 'movies'}]
# df_seg = spark.createDataFrame(d).groupBy('uid').agg(F.collect_set("segment")).collect()
# df_seg

# UPDATE DYNAMODB TABLE
# db = boto3.client("dynamodb")
# for row in df_seg:
#     db.update_item(TableName='user-profile',
#                    Key={'uid':{'N':row[0]}},
#                    UpdateExpression="ADD interests :segments_set",
#                    ExpressionAttributeValues={":segments_set":{"SS":row[1]}})


In [27]:
######### BATCH WRITING  #########
# dynamodb = boto3.resource('dynamodb')
# table = dynamodb.Table('user-profile')
           

# with table.batch_writer() as batch:
#     for row in df_seg:
#         print(row[0]+':'+str(row[1]))
#         batch.put_item(
#             Item={
#                 'uid':{'N':row[0]},
#                 'interests':{"SS":row[1]}            
            
#             }                            
           
#         )


In [2]:
import boto3
import findspark
findspark.init()
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import from_json, col

## UPDATE DYNAMODB TABLE
# db = boto3.client("dynamodb")
# for row in df_seg:
#     db.update_item(TableName='user-profile',
#                    Key={'uid':{'N':row[0]}},
#                    UpdateExpression="ADD interests :segments_set",
#                    ExpressionAttributeValues={":segments_set":{"SS":row[1]}})


# class SendToDynamoDB_ForeachWriter:
#   '''
#   Class to send a set of rows to DynamoDB.
#   When used with `foreach`, copies of this class is going to be used to write
#   multiple rows in the executor. See the python docs for `DataStreamWriter.foreach`
#   for more details.
#   '''

#   def open(self, partition_id, epoch_id):
#     # This is called first when preparing to send multiple rows.
#     # Put all the initialization code inside open() so that a fresh
#     # copy of this class is initialized in the executor where open()
#     # will be called.
#     #self.dynamodb = get_dynamodb()
#     return True

#   def process(self, row):
#     # This is called for each row after open() has been called.
#     # This implementation sends one row at a time.
#     # A more efficient implementation can be to send batches of rows at a time.
#     db = boto3.client("dynamodb")
#     db.update_item(TableName='user-profile',
#     Key={'uid':{'N':row[0]}},
#     UpdateExpression="ADD interests :segments_set",
#     ExpressionAttributeValues={":segments_set":{"SS":row[1]}})

#   def close(self, err):
#     # This is called after all the rows have been processed.
#     if err:
#       raise err


def sendToDynamoDB_simple(row):  
    db = boto3.client("dynamodb")
    db.update_item(TableName='user-profile',
    Key={'uid':{'N':row[0]}},
    UpdateExpression="ADD interests :segments_set",
    ExpressionAttributeValues={":segments_set":{"SS":row[1]}})
    pass

    
#val myschema = StructType().add("uid", IntegerType()).add("evttype", StringType()).add("segment", StringType())

schema = StructType([
    StructField('uid', StringType(), False),
    StructField('evttype', StringType(), False),
    StructField('segment', StringType(), False),
    ])

#spark.conf.set("spark.sql.shuffle.partitions", "1")

query = ( 
    spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "user-profile") \
    .option("startingOffsets", "earliest") \
    .load() \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema)) \
    .select( col("from_json(value).uid"),col("from_json(value).segment") ) \
    .groupBy("uid").agg(F.collect_set("segment").alias("interests")) \
    .writeStream \
    .foreach(sendToDynamoDB_simple) \
    .outputMode("complete") \
    .start()
      
)

#df.printSchema() 